In [1]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

data68755


In [2]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
!ls /home/aistudio/work

In [3]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
!mkdir /home/aistudio/external-libraries
!pip install beautifulsoup4 -t /home/aistudio/external-libraries

mkdir: cannot create directory ‘/home/aistudio/external-libraries’: File exists
Looking in indexes: https://mirror.baidu.com/pypi/simple/, https://mirrors.aliyun.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.9/147.9 kB 2.3 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip available: 22.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [4]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
sys.path.append('/home/aistudio/external-libraries')

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 

# 请使用PaddleHub实现十二生肖识别

In [5]:
import os
import pandas as pd
import zipfile
import random
import numpy as np
import paddle
import matplotlib.pyplot as plt
from PIL import Image
from PIL import ImageFile
from paddle.vision.transforms import Compose, Normalize,RandomHorizontalFlip
from paddle.io import Dataset

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized


In [6]:
# 安装并升级PaddleHub，使用百度源更稳定、更迅速
!pip install paddlehub==2.1 -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple, https://mirrors.aliyun.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.4/211.4 kB 4.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: paddlehub
    Found existing installation: paddlehub 2.3.0
    Uninstalling paddlehub-2.3.0:
      Successfully uninstalled paddlehub-2.3.0

[notice] A new release of pip available: 22.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [7]:
#数据解压缩
!cp /home/aistudio/data/data68755/signs.zip ./
!unzip -o signs.zip

In [8]:
import os

import paddle
import paddlehub as hub


class DemoDataset(paddle.io.Dataset):
    def __init__(self, transforms, num_classes=12, mode='train'):	
        # 数据集存放位置
        self.dataset_dir = "signs"  #dataset_dir为数据集实际路径，需要填写全路径
        self.transforms = transforms
        self.num_classes = num_classes
        self.mode = mode

        if self.mode == 'train':
            self.file = 'train.txt'
        elif self.mode == 'test':
            self.file = 'test.txt'
        else:
            self.file = 'valid.txt'
        
        self.file = os.path.join(self.dataset_dir , self.file)
        with open(self.file, 'r') as file:
            self.data = file.read().split('\n')[:-1]
            
    def __getitem__(self, idx):
        img_path, grt = self.data[idx].split(' ')
        
        im = self.transforms(img_path)#对图片进行预处理和增强
        return im, int(grt)


    def __len__(self):
        return len(self.data)

In [9]:
text = '''__all__ = ['CONFIG', 'get']

CONFIG = {
    'model_save_dir': "./output/zodiac",
    'num_classes': 12,
    'total_images': 7096,
    'epochs': 20,
    'batch_size': 32,
    'image_shape': [3, 224, 224],
    'LEARNING_RATE': {
        'params': {
            'lr': 0.00375             
        }
    },
    'OPTIMIZER': {
        'params': {
            'momentum': 0.9
        },
        'regularizer': {
            'function': 'L2',
            'factor': 0.000001
        }
    },
    'LABEL_MAP': [
        "ratt",
        "ox",
        "tiger",
        "rabbit",
        "dragon",
        "snake",
        "horse",
        "goat",
        "monkey",
        "rooster",
        "dog",
        "pig",
    ]
}

def get(full_path):
    for id, name in enumerate(full_path.split('.')):
        if id == 0:
            config = CONFIG
        
        config = config[name]
    
    return config
'''
fid = open('config.py', 'w')
fid.write(text)
fid.close()

In [10]:
import io
import os
from PIL import Image
from config import get


# 数据集根目录
DATA_ROOT = 'signs'

# 标签List
LABEL_MAP = get('LABEL_MAP')

# 标注生成函数
def generate_annotation(mode):
    # 建立标注文件
    with open('{}/{}.txt'.format(DATA_ROOT, mode), 'w') as f:
        # 对应每个用途的数据文件夹，train/valid/test
        train_dir = '{}/{}'.format(DATA_ROOT, mode)

        # 遍历文件夹，获取里面的分类文件夹
        for path in os.listdir(train_dir):
            # 标签对应的数字索引，实际标注的时候直接使用数字索引
            label_index = LABEL_MAP.index(path)

            # 图像样本所在的路径
            image_path = '{}/{}'.format(train_dir, path)

            # 遍历所有图像
            for image in os.listdir(image_path):
                # 图像完整路径和名称
                image_file = '{}/{}'.format(image_path, image)
                
                try:
                    # 验证图片格式是否ok
                    with open(image_file, 'rb') as f_img:
                        image = Image.open(io.BytesIO(f_img.read()))
                        image.load()
                        
                        if image.mode == 'RGB':
                            f.write('{} {}\n'.format(image_file, label_index))
                except:
                    continue


generate_annotation('train')  # 生成训练集标注文件
generate_annotation('valid')  # 生成验证集标注文件
generate_annotation('test')   # 生成测试集标注文件

In [11]:
import paddlehub.vision.transforms as T

transforms = T.Compose(
        [T.Resize((256, 256)),#图像缩放
         T.CenterCrop(224),#中心剪裁从图像的中心抠图,大小为224x224
         T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])],#这一组平均值是从imagenet训练集中抽样算出来的这一组平均值是从imagenet训练集中抽样算出来的
        to_rgb=True)

peach_train = DemoDataset(transforms)
peach_validate =  DemoDataset(transforms, mode='val')

In [12]:
#安装预训练模型
! hub install resnet50_vd_imagenet_ssld==1.1.0

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized
Download https://bj.bcebos.com/paddlehub/paddlehub_dev/resnet50_vd_imagenet_ssld.tar.gz
[##################################################] 100.00%
Decompress

In [13]:
import paddlehub as hub

model = hub.Module(name='resnet50_vd_imagenet_ssld', label_list=[ "ratt",
        "ox",
        "tiger",
        "rabbit",
        "dragon",
        "snake",
        "horse",
        "goat",
        "monkey",
        "rooster",
        "dog",
        "pig"])

W0512 16:59:13.824287   114 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 12.0, Runtime API Version: 11.2
W0512 16:59:13.829116   114 gpu_resources.cc:91] device: 0, cuDNN Version: 8.2.


load pretrained parameters success


In [14]:
from paddlehub.finetune.trainer import Trainer

import paddle

optimizer = paddle.optimizer.Adam(learning_rate=0.001, parameters=model.parameters())
trainer = Trainer(model, optimizer, checkpoint_dir='img_classification_ckpt', use_gpu=True) 
trainer.train(peach_train, epochs=10, batch_size=16, eval_dataset=peach_validate, save_interval=1)

[2024-05-12 16:59:16,536] [    INFO] - PaddleHub model checkpoint loaded. current_epoch=1 [acc=0.3099]
libpng warning: iCCP: known incorrect sRGB profile
[2024-05-12 16:59:21,429] [   TRAIN] - Epoch=2/10, Step=10/444 loss=1.9783 acc=0.2688 lr=0.001000 step/sec=2.41 | ETA 00:30:39
[2024-05-12 16:59:25,534] [   TRAIN] - Epoch=2/10, Step=20/444 loss=1.8547 acc=0.3375 lr=0.001000 step/sec=2.44 | ETA 00:30:31
[2024-05-12 16:59:31,205] [   TRAIN] - Epoch=2/10, Step=30/444 loss=2.0335 acc=0.3063 lr=0.001000 step/sec=1.76 | ETA 00:34:19
[2024-05-12 16:59:34,592] [   TRAIN] - Epoch=2/10, Step=40/444 loss=2.0315 acc=0.2937 lr=0.001000 step/sec=2.95 | ETA 00:32:00
[2024-05-12 16:59:37,730] [   TRAIN] - Epoch=2/10, Step=50/444 loss=1.8500 acc=0.3688 lr=0.001000 step/sec=3.19 | ETA 00:30:15
libpng warning: iCCP: known incorrect sRGB profile
[2024-05-12 16:59:41,589] [   TRAIN] - Epoch=2/10, Step=60/444 loss=1.7723 acc=0.4000 lr=0.001000 step/sec=2.59 | ETA 00:29:58
[2024-05-12 16:59:44,343] [   TRA

In [19]:
import paddle
import paddlehub as hub

result = model.predict(['signs/valid/pig/00000166.jpg'])
print(result)

[{'pig': 0.95014805}]
